# Solving MaxCut with KOPPU

This notebook demonstrates how to solve the Maximum Cut (MaxCut) problem using the KOPPU OPU. We will generate a random graph, map it to the OPU, and visualize the solution.

In [ ]:
import pykoppu as pk
import networkx as nx
import matplotlib.pyplot as plt

# 1. Generate a Random Graph
print("Generating Graph...")
G = nx.erdos_renyi_graph(n=20, p=0.3, seed=42)
pos = nx.spring_layout(G)

# 2. Create MaxCut Problem
print("Mapping to OPU...")
problem = pk.problems.graph.MaxCut(G)

# 3. Solve
print("Solving...")
# We set the simulation duration to 500ms using the 't' parameter
process = pk.oos.Process(problem, backend='cpu', t=500)
result = process.run()

# 4. Display OPU Dynamics
print("Visualizing OPU Dynamics...")
result.plot()

In [ ]:
# 5. Analyse and Display Solution
print("Visualizing Graph Solution...")
# We use a threshold of 0.3 to determine if a neuron has 'fired' (state=1)
# You can run this cell multiple times with different thresholds to explore the solution space
problem.plot(result, threshold=0.3)